# Exploring DynamoDB Streams

Streams in DynamoDB will update us when new events occur, which we'll need for supervision.  I'll use this to test them.

I've added the below code to our database creation, which should then generate streams.  Now I need to try and read them.

'''

        StreamSpecification = {
            'StreamEnabled': True,
            'StreamViewType': 'NEW_AND_OLD_IMAGES'
        }
    )
'''

In [ ]:
import os

from dotenv import load_dotenv
import boto3

load_dotenv() 

In [ ]:

# Initialize a DynamoDB Streams client pointing to your local setup
client = boto3.client('dynamodbstreams', endpoint_url="http://localhost:8000", region_name='eu-west-2',aws_access_key_id=os.getenv('DB_ACCESS_KEY_ID'), aws_secret_access_key=os.getenv('DB_SECRET_ACCESS_KEY'))

# Get the Stream ARN
response = client.list_streams(TableName='caddyMessages')
response

In [ ]:
stream_arn = response['Streams'][0]['StreamArn']
stream_arn

In [ ]:

# Get a Shard ID
shard_id = client.describe_stream(StreamArn=stream_arn)['StreamDescription']['Shards'][0]['ShardId']
shard_id

In [ ]:

# Get Shard Iterator
iterator = client.get_shard_iterator(
    StreamArn=stream_arn,
    ShardId=shard_id,
    ShardIteratorType='LATEST'  # Can also use 'TRIM_HORIZON', 'AT_SEQUENCE_NUMBER', etc.
)['ShardIterator']
iterator

In [ ]:

# Get Records
records_response = client.get_records(ShardIterator=iterator)
records = records_response['Records']
records

In [ ]:
while True:
    records_response = client.get_records(ShardIterator=iterator)
    records = records_response['Records']
    for record in records:
        print(record)
    iterator = records_response['NextShardIterator']


So here is an event:
``````
{'eventID': '3abae425-6b6a-4bec-8ee3-c0451642e7e8', 'eventName': 'INSERT', 'eventVersion': '1.1', 'eventSource': 'aws:dynamodb', 'awsRegion': 'ddblocal', 'dynamodb': {'ApproximateCreationDateTime': datetime.datetime(2023, 10, 27, 16, 11, tzinfo=tzlocal()), 'Keys': {'conversationId': {'S': 'None'}, 'messageId': {'S': '1093c7eb-5d51-4e40-976e-83b789a29f9e'}}, 'NewImage': {'llm_prompt': {'S': 'What do you think of Citizens Advice?'}, 'conversationId': {'S': 'None'}, 'messageId': {'S': '1093c7eb-5d51-4e40-976e-83b789a29f9e'}, 'text': {'S': 'What do you think of Citizens Advice?'}, 'timestamp': {'S': '2023-10-27T15:11:39.464543'}}, 'SequenceNumber': '000000000000000000005', 'SizeBytes': 249, 'StreamViewType': 'NEW_AND_OLD_IMAGES'}}
```'''```